In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from anchor.visualize import MODALITY_TO_COLOR, MODALITY_ORDER, MODALITY_PALETTE
modality_order = MODALITY_ORDER

sns.set(style='ticks', context='paper', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

%matplotlib inline

%load_ext autoreload
%autoreload 2

folder = 'pdfs'#'/home/obotvinnik/Dropbox/figures2/singlecell_pnm/figure4_voyages'

import flotilla

flotilla_dir = '/projects/ps-yeolab/obotvinnik/flotilla_projects'
study = flotilla.embark('singlecell_pnm_figure4_voyages', flotilla_dir=flotilla_dir)

/home/obotvinnik/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:810: UserWarning: Found matplotlib configuration in ~/.matplotlib/. To conform with the XDG base directory standard, this configuration location has been deprecated on Linux, and the new location is now /home/obotvinnik/.config/matplotlib/. Please move your configuration there to ensure that matplotlib will continue to find it in the future.
  _get_xdg_config_dir())
/home/obotvinnik/anaconda/lib/python2.7/site-packages/flotilla/__init__.py:55: DtypeWarning: Columns (11,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  load_species_data=load_species_data)


2016-03-24 14:47:51	Reading datapackage from /projects/ps-yeolab/obotvinnik/flotilla_projects/singlecell_pnm_figure4_voyages/datapackage.json
2016-03-24 14:47:51	Parsing datapackage to create a Study object
2016-03-24 14:48:36	Initializing Study
2016-03-24 14:48:36	Initializing Predictor configuration manager for Study
2016-03-24 14:48:36	Predictor ExtraTreesClassifier is of type <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
2016-03-24 14:48:36	Added ExtraTreesClassifier to default predictors
2016-03-24 14:48:36	Predictor ExtraTreesRegressor is of type <class 'sklearn.ensemble.forest.ExtraTreesRegressor'>
2016-03-24 14:48:36	Added ExtraTreesRegressor to default predictors
2016-03-24 14:48:36	Predictor GradientBoostingClassifier is of type <class 'sklearn.ensemble.gradient_boosting.GradientBoostingClassifier'>
2016-03-24 14:48:36	Added GradientBoostingClassifier to default predictors
2016-03-24 14:48:36	Predictor GradientBoostingRegressor is of type <class 'sklearn.ensemble.gra

In [ ]:
figure_folder = '{}/from_messy_to_extremes'.format(folder)
! mkdir $figure_folder

## Violinplots

In [ ]:

violinplot_folder = '{}/violinplots'.format(figure_folder)
! mkdir $violinplot_folder

for event in ipsc_lower_triangle_in_mn_waypoints.index:
    gene_name = study.splicing.feature_data['gene_name'][event]
    study.plot_event(event, sample_subset='~outlier')
    fig = plt.gcf()
    fig.savefig('{}/{}_{}.pdf'.format(violinplot_folder, gene_name, event))

## Voyageplots

In [ ]:
voyageplot_folder = '{}/voyageplots'.format(figure_folder)
! mkdir $voyageplot_folder

for event in ipsc_lower_triangle_in_mn_waypoints.index:
    gene_name = study.splicing.feature_data['gene_name'][event]
    fig, ax = plt.subplots(figsize=(3, 3))
    voyageplot(waypoints_index, event, study.phenotype_to_color, study.phenotype_to_marker, 
               study.phenotype_order, ax=ax, xlabel='~0', ylabel='~1')
#     fig = plt.gcf() 
    sns.despine()
    title = '{}\n{}'.format(gene_name, '\n'.join(event.split('@')))
    ax.set_title(title, fontsize=10)
    fig.tight_layout()
    fig.savefig('{}/{}_{}.pdf'.format(voyageplot_folder, gene_name, event))